<a href="https://colab.research.google.com/github/bhavikngala/object_detection_in_point_clouds/blob/dev/pixor_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
!pwd
!mkdir -p data
!cp -r /gdrive/My\ Drive/KITTI_BEV/* ./data


/content


In [1]:
!ls ./data/training/velodyne | wc -l
!ls ./data/training/label_2 | wc -l
!ls ./data/training/calib | wc -l

7481
7481
7482


In [23]:
%cd /content
!git clone --branch doogal https://<USERNAME>:<PASSWORD>@github.com/bhavikngala/object_detection_in_point_clouds.git
!mv object_detection_in_point_clouds code
%cd ./code
!git pull origin doogal

/content
Cloning into 'object_detection_in_point_clouds'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 1869 (delta 67), reused 69 (delta 31), pack-reused 1760
Receiving objects: 100% (1869/1869), 6.09 MiB | 27.48 MiB/s, done.
Resolving deltas: 100% (1239/1239), done.
/content/code
From https://github.com/bhavikngala/object_detection_in_point_clouds
 * branch            doogal     -> FETCH_HEAD
Already up to date.


In [3]:
!pwd
%cd ./code

/content
/content/code


In [0]:
!mkdir -p models
!mkdir -p loss

### Custom trainer

In [0]:
import torch
from torch.utils.data import DataLoader

import os
import traceback
import argparse
try:
  from tensorboardX import SummaryWriter
  log = True
except:
  log = False

import datautils.dataloader_v3 as dataloader
import config as cnf
import misc
import networks.networks as networks
import model_groomers as mg
import lossUtils as lu

In [0]:
class CustomGroomer(mg.ModelTrainer):

  def __init__(self, log, logDir, modelFilename, **kwargs):
    if log: self.writer = SummaryWriter(logDir)
    self.iter = 0
    self.logDir = logDir
    self.modelFilename = modelFilename
    self.clip_value = kwargs['clip_value']
    self.accumulationSteps = kwargs['accumulationSteps']
    self.lrRange2 = kwargs['lrRange2']
    self.momentumRange2 = kwargs['momentumRange2']

  def train(self, device, log=False):
    if self.loader is None:
      print('data loader is undefined')
      quit()

    subBatchCounter = 0
    for epoch in range(1, self.epochs+1):
      epochValues = []

      if self.oneCycleLearning:
        if epoch > self.stepSize*2:
          self.setLrRangeStepSize(self.lrRange2, self.momentumRange2, self.stepSize)
        self.oneCycleStep(epoch)
      else:
        self.scheduler.step()

      batch_size = self.loader.batch_size
      self.model.zero_grad()
      for batchId, data in enumerate(self.loader):
        lidar, targetClass, targetLoc, filenames = data

        # ignore last batch
        if lidar.size(0) != batch_size:
          continue

        lidar = lidar.cuda(device, non_blocking=True)
        targetClass = [c.contiguous().cuda(device, non_blocking=True) for c in targetClass]
        targetLoc = [loc.contiguous().cuda(device, non_blocking=True) for loc in targetLoc]

        predictedClass, predictedLoc = self.model(lidar)

        claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples \
         = self.lossFunction(predictedClass, predictedLoc, targetClass, targetLoc, self.alpa1, self.beta1)

        if trainLoss is not None:
          trainLoss = trainLoss/self.accumulationSteps
          trainLoss.backward()
          subBatchCounter += 1

        if subBatchCounter==self.accumulationSteps:
          if self.clip_value:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_value)
          self.optim.step()
          self.model.zero_grad()
          subBatchCounter = 0

    
        epochValues.append((claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples))
        if log: self.iterLogger((claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples))
          
      self.epochLogger(epochValues, epoch, log)
      self.saveModel(self.modelFilename)

  def val(self):
    pass

  def iterLogger(self, values):
    claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples = \
      values
    self.writer.add_scalar('data/classification_loss', claLoss, self.iter)
    if posClaLoss is not None:
      self.writer.add_scalar('data/pos_classification_loss', posClaLoss, self.iter)
      self.writer.add_scalar('data/localization_loss', locLoss, self.iter)
      self.writer.add_scalar('data/mean_pos_sample_confidence', meanConfidence, self.iter)
    self.writer.add_scalar('data/neg_classification_loss', negClaLoss, self.iter)
    self.writer.add_scalar('data/train_loss', trainLoss, self.iter)
    self.writer.add_scalar('data/mean_pt', overallMeanConfidence, self.iter)
    self.writer.add_scalar('data/pos_samples', numPosSamples, self.iter)
    self.writer.add_scalar('data/neg_samples', numNegSamples, self.iter)
    self.iter += 1

  def epochLogger(self, epochValues, epoch, log=False):
    pC, nC, locL, mC, mPT, nP, nN = 0, 0, 0, 0, 0, 0, 0
    for i in range(len(epochValues)):
      claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples = \
        epochValues[i]
      if posClaLoss is not None:
        pC = pC + posClaLoss*numPosSamples
        locL = locL + locLoss*numPosSamples
        mC = mC + meanConfidence*numPosSamples
      nC = nC + negClaLoss*numNegSamples
      mPT = mPT + overallMeanConfidence*(numPosSamples+numNegSamples)
      nP += numPosSamples
      nN += numNegSamples

    pC /= nP
    nC /= nN
    locL /= nP
    mC /= nP
    mPT /= (nP+nN)

    if log:
      self.writer.add_scalar('train/epoch_classification_loss', pC+nC, epoch)
      self.writer.add_scalar('train/epoch_pos_classification_loss', pC, epoch)
      self.writer.add_scalar('train/epoch_neg_classification_loss', nC, epoch)
      self.writer.add_scalar('train/epoch_localization_loss', locL, epoch)
      self.writer.add_scalar('train/epoch_train_loss', pC+nC+locL, epoch)
      self.writer.add_scalar('train/epoch_mean_pos_sample_confidence', mC, epoch)
      self.writer.add_scalar('train/epoch_mean_pt', mPT, epoch)
    else:
      cl = pC + nC
      tl = pC+nC+locL
      print(f'epoch: {epoch} | CL: {cl} | PC: {pC} | NC: {nC} | LL: {locL} | TL: {tl} | MPC: {mC} | MC: {mPT}')

  def exportLogs(self, filename):
    # export scalar data to JSON for external processing
    self.writer.export_scalars_to_json(filename)
    self.writer.close()

  def setLoaders(self):
    pass

In [0]:
def main(args):
  # data loaders
  trainLoader = DataLoader(
    dataloader.KittiDataset(cnf, args, 'train'),
    batch_size = cnf.batchSize, shuffle=True, num_workers=3,
    collate_fn=dataloader.customCollateFunction, pin_memory=True
  )

  # define model
  model = networks.PointCloudDetector2(
    cnf.res_block_layers,
    cnf.up_sample_layers,
    cnf.deconv)

  modelTrainer = CustomGroomer(False, cnf.logDir, args.model_file,
    clip_value=cnf.clip_value,
    accumulationSteps=cnf.accumulationSteps,
    lrRange2 = cnf.lrRange2,
    momentumRange2 = cnf.momentumRange2)
  modelTrainer.setDataloader(trainLoader)
  modelTrainer.setEpochs(cnf.epochs)
  modelTrainer.setModel(model)
  modelTrainer.setDataParallel(args.multi_gpu)
  modelTrainer.copyModel(cnf.device)
  modelTrainer.setOptimizer('sgd', cnf.slr, cnf.momentum, cnf.decay)
  if cnf.cycleLearn:
    modelTrainer.setLrRangeStepSize(cnf.lrRange, cnf.momentumRange, cnf.stepSize)
  else:
    modelTrainer.setLRScheduler(cnf.lrDecay, cnf.milestones)

  if os.path.isfile(args.model_file):
    modelTrainer.loadModel(args.model_file)

  modelTrainer.setLossFunction(lu.computeLoss, cnf.alpha1, cnf.beta1)
  modelTrainer.train(cnf.device, cnf.pretrainCla)
  modelTrainer.exportLogs(cnf.logJSONFilename)

In [11]:
MODEL_FILE = './models/p1.pth'
cnf.rootDir = './../data'
cnf.calTrain = './../data/training/calib'
print(cnf.device)

parser = argparse.ArgumentParser(description='Train network')
parser.add_argument('-f', '--model-file', default=None, help='used to set different model file name other than default one')
parser.add_argument('-m', '--multi-gpu', action='store_true', help='use multiple gpus?')
parser.add_argument('--full-train-set', action='store_true', help='use entire training set or split into train val set')
parser.add_argument('--plot-img', action='store_true', help='plot on img; used in validateNetwork.py')
args = parser.parse_args()
args.model_file = MODEL_FILE

try:
  main(args)
except Exception as e:
  with open('./error.txt', 'w') as f:
    print(traceback.format_exc())
    f.write(traceback.format_exc())

cuda
Traceback (most recent call last):
  File "<ipython-input-11-24559fb0120c>", line 15, in <module>
    main(args)
  File "<ipython-input-10-5e74f4a9afca>", line 35, in main
    modelTrainer.train(cnf.device, cnf.pretrainCla)
  File "<ipython-input-6-95c4563b181e>", line 31, in train
    for batchId, data in enumerate(self.loader):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 637, in __next__
    return self._process_next_batch(batch)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 658, in _process_next_batch
    raise batch.exc_type(batch.exc_msg)
KeyError: 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop\n    samples = collate_fn([dataset[i] for i in batch_indices])\n  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>\n    samples = collate_fn([dataset[i] for i 

In [0]:
!tail ./../data/training/calib/000000.txt